In [ ]:
import os

import numpy as np
import xarray as xr
import rioxarray as rxr

import gval

from mosaic_inundation import Mosaic_inundation
from inundate_gms import Inundate_gms

In [ ]:
fim_dir = '/outputs/12020001'
benchmark_flows = f'{fim_dir}/ble_huc_12020001_flows_500yr.csv'
gms_workers = 1
huc = ['12020001']
predicted_raster_path = '/outputs/inundation.tif'
verbose = False
log_file = f'{fim_dir}/gms.log'
elev_raster_ndv = -9999
map_file = Inundate_gms(hydrofabric_dir=os.path.dirname(fim_dir),
                        forecast=benchmark_flows,
                        num_workers=gms_workers,
                        hucs=huc,
                        inundation_raster=predicted_raster_path,
                        inundation_polygon=None,
                        depths_raster=None,
                        verbose=verbose,
                        log_file=log_file,
                        output_fileNames=None)

mosaic_raster = Mosaic_inundation(map_file,
                                    mosaic_attribute='inundation_rasters',
                                    mosaic_output=predicted_raster_path,
                                    mask=os.path.join(fim_dir, 'wbd.gpkg'),
                                    unit_attribute_name='huc8',
                                    nodata=elev_raster_ndv,
                                    workers=1,
                                    remove_inputs=True,
                                    subset=None,
                                    verbose=verbose)

In [ ]:
candidate = rxr.open_rasterio(mosaic_raster, mask_and_scale=True)
benchmark = rxr.open_rasterio('/outputs/ble_huc_12020001_extent_500yr.tif', mask_and_scale=True)

In [ ]:
candidate.data = xr.where(candidate < 0, 0, candidate)
candidate.data = xr.where(candidate > 0, 1, candidate)

In [ ]:
target_map, cross_tab, metric_df = candidate.gval.categorical_compare(benchmark,
                                                                      positive_categories=[1],
                                                                      negative_categories=[0])

In [ ]:
target_map.plot()

In [ ]:
cross_tab

In [ ]:
metric_df